In [1]:
exiftool_path = r"C:\Windows\exiftool.exe"
input_dir = r"D:\Trial\Sudasari B\Sudasari B2\Test_14022024"
dest_dir = r"G:\Guzzler_data\Sorted_guzzler_data\2020\SudasariACD\A\20200321_165610__to__20200505_124728"
Station = "SudasariACD"
Camera = "B"
latitude = 26.723574  # Replace with your latitude
longitude = 70.634269  # Replace with your longitude

In [2]:
#### Function for find corrupt files
import os

def list_corrupt_files_in_directory(directory):
    corrupt_files=[]
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                # Use the os.path.getsize() method to check if the file is empty or very small
                file_size = os.path.getsize(file_path)
                if file_size < 10:  # Adjust the threshold as needed
                    corrupt_files.append(file_path)
                    print(f"Corrupt file: {file_path}")
                else:
                    continue
            except Exception as e:
                # Handle other exceptions that might occur
                print(f"Error processing file {file_path}: {e}")
    return corrupt_files

In [3]:
#### Find and delete corrupt photos

corrupt_files=list_corrupt_files_in_directory(input_dir)
if len(corrupt_files) > 0:
    for c in corrupt_files:
        os.remove(c)
print(f"Corrupt files : {len(corrupt_files)} images removed")

Corrupt files : 0 images removed


['F:\\Pred_Estimation_2023\\PPC\\331_117_3_4298\\1_13_2024\\100CUDDY\\I_00035b.JPG',
 'F:\\Pred_Estimation_2023\\PPC\\331_117_3_4298\\1_13_2024\\100CUDDY\\I_00035c.JPG',
 'F:\\Pred_Estimation_2023\\PPC\\331_117_3_4298\\1_13_2024\\100CUDDY\\I_00035d.JPG',
 'F:\\Pred_Estimation_2023\\PPC\\331_117_3_4298\\1_13_2024\\100CUDDY\\I_00035e.JPG',
 'F:\\Pred_Estimation_2023\\PPC\\331_117_3_4298\\1_13_2024\\100CUDDY\\I_00137c.JPG',
 'F:\\Pred_Estimation_2023\\PPC\\331_117_3_4298\\1_13_2024\\100CUDDY\\I_00138b.JPG',
 'F:\\Pred_Estimation_2023\\PPC\\331_117_3_4298\\1_13_2024\\100CUDDY\\I_00138c.JPG',
 'F:\\Pred_Estimation_2023\\PPC\\331_117_3_4298\\1_13_2024\\100CUDDY\\I_00138d.JPG',
 'F:\\Pred_Estimation_2023\\PPC\\331_117_3_4298\\1_13_2024\\100CUDDY\\I_00138e.JPG',
 'F:\\Pred_Estimation_2023\\PPC\\331_117_3_4298\\1_13_2024\\100CUDDY\\I_00140c.JPG',
 'F:\\Pred_Estimation_2023\\PPC\\331_117_3_4298\\1_13_2024\\100CUDDY\\I_00140d.JPG',
 'F:\\Pred_Estimation_2023\\PPC\\331_117_3_4298\\1_13_2024\\100CU

In [1]:
#### Fucntions for fast renaming and copying

import datetime
import pandas as pd
import exifread
import os
from datetime import timedelta
import shutil
from concurrent.futures import ThreadPoolExecutor
import math
import pyfastcopy
import gc

def list_files_in_directory(directory):
    file_paths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg','.csv','.json')):
                file_path = os.path.join(root, file)
                file_paths.append(file_path)
            else:
                continue
    return file_paths

def convert_datetime(dt):
    try:
        dt = pd.to_datetime(dt, format='%Y-%m-%d %H-%M-%S')
        return dt.strftime('%Y%m%d_%H%M%S')
    except pd.errors.OutOfBoundsDatetime:
        return 'OutOfBoundsDatetime'

def clean_path(path):
    return os.path.normpath(path)

def process_image(image_path):
    with open(image_path, 'rb') as image_file:
        tags = exifread.process_file(image_file, details=False)
        
        directory = os.path.dirname(image_path)
        filename = os.path.basename(image_path)
        filetype_extension = os.path.splitext(filename)[1]
        make = tags.get('Image Make', 'N/A')
        model = tags.get('Image Model', 'N/A')
        datetime_original = tags.get('EXIF DateTimeOriginal', 'N/A')
        
        return {
            'SourceFile': image_path,
            'Directory': directory,
            'FileName': filename,
            'FileTypeExtension': filetype_extension,
            'Make': make,
            'Model': model,
            'DateTimeOriginal': datetime_original
        }

def read_exif(image_dir):
    file_paths = list_files_in_directory(image_dir)
    print(len(file_paths))
    with ThreadPoolExecutor(20) as executor:  # Adjust max_workers as needed
        image_metadata_list = list(executor.map(process_image, file_paths))
    exif_info = pd.DataFrame(image_metadata_list)
    return exif_info

def create_new_filenames(exif_info):
    exif_info = exif_info[exif_info["DateTimeOriginal"] != "N/A"]
    highest_subfolder_number = get_highest_subfolder_number(dest_dir)
    exif_info['Station'] = Station
    exif_info['Camera'] = Camera
    exif_info['DateTimeOriginal'] = pd.to_datetime(exif_info['DateTimeOriginal'], format='%Y:%m:%d %H:%M:%S')
    exif_info['FormattedDateTime'] = exif_info['DateTimeOriginal'].apply(convert_datetime)
    exif_info = exif_info.sort_values(by=['Station', 'Camera', 'DateTimeOriginal']).reset_index(drop=True)
    exif_info['diff'] = exif_info.groupby(['Station', 'Camera'])['DateTimeOriginal'].diff()
    exif_info['image_number']=exif_info.groupby(['Station','Camera']).cumcount()+1
    exif_info['Directory'] = exif_info['Directory'].apply(clean_path)
    exif_info['SourceFile'] = exif_info['SourceFile'].apply(clean_path)
    exif_info['Dest_subfolder_number'] = (highest_subfolder_number + exif_info['image_number'].apply(lambda x: math.ceil(x / 10000))).astype(str)
    
    subfolders=[]
    min_dates=[]
    max_dates=[]
    for d in exif_info["Dest_subfolder_number"].unique():
        subfolders.append(d)
        temp_renaming_table = exif_info.loc[exif_info["Dest_subfolder_number"] == d]  
        min_date = datetime.datetime.strftime(min(temp_renaming_table.FormattedDateTime.apply(lambda x: datetime.datetime.strptime(x,'%Y%m%d_%H%M%S'))),'%Y%m%d_%H%M%S')
        min_dates.append(min_date)
        max_date = datetime.datetime.strftime(max(temp_renaming_table.FormattedDateTime.apply(lambda x: datetime.datetime.strptime(x,'%Y%m%d_%H%M%S'))),'%Y%m%d_%H%M%S')
        max_dates.append(max_date)
    subfolder_intervals_df = pd.DataFrame({"Dest_subfolder_number" : subfolders, "min_date" : min_dates, "max_date" : max_dates})
    exif_info = exif_info.merge(subfolder_intervals_df, how = "left")
    exif_info['Dest_Directory'] = (dest_dir + "\\" + exif_info['min_date'] + "__to__" + exif_info["max_date"]).apply(clean_path)
    #exif_info['Dest_Directory'] = (dest_dir + "\\" + exif_info["Dest_subfolder_number"]).apply(clean_path)

    ### Add sequence number
    threshold = timedelta(seconds=1)
    Sequence = []
    for i in range(len(exif_info)):
        diff = exif_info['diff'][i]
        if pd.isna(diff) or diff > threshold:
            sequence = 1
        else:
            sequence = Sequence[i - 1] + 1
        Sequence.append(sequence)
    exif_info['Sequence'] = Sequence

    ### Construct new filename
    exif_info['FileNameNew'] = exif_info['Station'] + '_' + exif_info['Camera'] + '_' + exif_info['FormattedDateTime'] + '(' + exif_info['Sequence'].astype(str) + ')' + exif_info['FileTypeExtension']
    exif_info['DestFile'] = (exif_info['Dest_Directory'] + "\\" + exif_info['FileNameNew']).apply(clean_path)
    
    return exif_info

def get_highest_subfolder_number(dest_dir):
    # Get existing subdirectories
    if os.path.isdir(dest_dir):
        existing_subdirs = [d for d in os.listdir(dest_dir) if os.path.isdir(os.path.join(dest_dir, d))]
        subfolder_numbers = [int(dir) for dir in existing_subdirs]
        try:
            max_number = max(subfolder_numbers)
        except:
            max_number = 0
    else:
        max_number = 0
    # Return the highest subfolder number or 0 if none exist
    return max_number

def copy_images_batch(table, batch_size=1000):
    src_files=table['SourceFile']
    dest_files=table['DestFile']
    with ThreadPoolExecutor(20) as exe:
        for i in range(0, len(src_files), batch_size):
            src_batch = src_files[i:i + batch_size]
            dest_batch = dest_files[i:i + batch_size]
            
            batch_tasks = [exe.submit(shutil.copy, src, dest) for src, dest in zip(src_batch, dest_batch)]
            # Wait for all tasks in the batch to complete before proceeding to the next batch
            _ = [task.result() for task in batch_tasks]
            print(f"First {i+1 * 1000} images copied at {datetime.datetime.now()}")
            gc.collect()

In [5]:
start = datetime.datetime.now()
###Create Renaming Table
exif = read_exif(input_dir)
renaming_table=create_new_filenames(exif)
print(f"Renaming table created in {datetime.datetime.now() - start}")

### Copy and rename in batches, based on renaming table
unique_directories = set(renaming_table['Dest_Directory'])
for d in unique_directories:
    if not os.path.exists(d):
        os.makedirs(d)
copy_images_batch(renaming_table)
print(f"Renaming and copying completed in {datetime.datetime.now() - start}")

147
Renaming table created in 0:00:00.459744
First 1000 images copied at 2024-03-02 17:10:54.876656
Renaming and copying completed in 0:00:01.082353


In [6]:
exif

,SourceFile,Directory,FileName,FileTypeExtension,Make,Model,DateTimeOriginal
0,D:\Trial\Sudasari B\Sudasari B2\Test_14022024\...,D:\Trial\Sudasari B\Sudasari B2\Test_14022024\A,IMG_0001.JPG,.JPG,RECONYX,HC500 HYPERFIRE,2023:05:23 19:35:00
1,D:\Trial\Sudasari B\Sudasari B2\Test_14022024\...,D:\Trial\Sudasari B\Sudasari B2\Test_14022024\A,IMG_0002.JPG,.JPG,RECONYX,HC500 HYPERFIRE,2023:05:23 19:36:00
2,D:\Trial\Sudasari B\Sudasari B2\Test_14022024\...,D:\Trial\Sudasari B\Sudasari B2\Test_14022024\A,IMG_0003.JPG,.JPG,RECONYX,HC500 HYPERFIRE,2023:05:23 19:37:00
3,D:\Trial\Sudasari B\Sudasari B2\Test_14022024\...,D:\Trial\Sudasari B\Sudasari B2\Test_14022024\A,IMG_0004.JPG,.JPG,RECONYX,HC500 HYPERFIRE,2023:05:23 19:38:00
4,D:\Trial\Sudasari B\Sudasari B2\Test_14022024\...,D:\Trial\Sudasari B\Sudasari B2\Test_14022024\A,IMG_0005.JPG,.JPG,RECONYX,HC500 HYPERFIRE,2023:05:23 19:39:00
...,...,...,...,...,...,...,...
142,D:\Trial\Sudasari B\Sudasari B2\Test_14022024\...,D:\Trial\Sudasari B\Sudasari B2\Test_14022024\B,IMG_0225.JPG,.JPG,RECONYX,HC500 HYPERFIRE,2023:01:29 11:18:48
143,D:\Trial\Sudasari B\Sudasari B2\Test_14022024\...,D:\Trial\Sudasari B\Sudasari B2\Test_14022024\B,IMG_0226.JPG,.JPG,RECONYX,HC500 HYPERFIRE,2023:01:29 11:18:48
144,D:\Trial\Sudasari B\Sudasari B2\Test_14022024\...,D:\Trial\Sudasari B\Sudasari B2\Test_14022024\B,IMG_0227.JPG,.JPG,RECONYX,HC500 HYPERFIRE,2023:01:29 11:18:59
145,D:\Trial\Sudasari B\Sudasari B2\Test_14022024\...,D:\Trial\Sudasari B\Sudasari B2\Test_14022024\B,IMG_0228.JPG,.JPG,RECONYX,HC500 HYPERFIRE,2023:01:29 11:19:00


In [7]:
#### Functions for geotagging

import subprocess
import exiftool
import datetime

def add_gps_info(image_dir):
    command = [exiftool_path, '-GPSLatitude=' + str(latitude), '-GPSLongitude=' + str(longitude), '-overwrite_original','-r',image_dir]
    try:
        subprocess.run(command, check=True)
        print(f"GPS information added to {image_dir} successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Error while processing {image_dir}: {e}")

In [8]:
### Geotag
start = datetime.datetime.now()
for d in unique_directories:
    add_gps_info(d)
end= datetime.datetime.now()
print(f"Geotagging completed in {end-start}")

GPS information added to D:\Trial\Sudasari B\Sudasari B2\Test_Dest_14022024\20230129_111729__to__20230523_222702 successfully.
Geotagging completed in 0:00:03.621962


In [8]:
### Run BRP_Anipredictor

from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from functions import *
import torch
from tqdm.notebook import tqdm
import gc
from Megadetector import *

try:
    print(f"Models loaded : {areModelsLoaded}")
except:
    areModelsLoaded = False
log = {}
now = datetime.now()

inp = "YES"

## CHECK FOR GPU

gpus = tf.config.experimental.list_physical_devices('GPU')
cpu = tf.config.experimental.list_physical_devices("CPU")

if gpus:
    print("GPU available")
    gpu_name = torch.cuda.get_device_name()
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            
    except:
        pass
else:
    print("No GPUs available.")
    
## GET SUBDIRECTORIES

sub_dirs = []
for dirpath, dirnames, filenames in os.walk(dest_dir):
    if not "Cropped_images" in dirpath and not dest_dir == dirpath: # 'not "Cropped_images" in dirnames and' at the start if required
        if dirpath.split("\\")[-1] not in ["Cropped_images","Chinkara","Nilgai", "Cattle","GIB", "Goat_Sheep", "Hare", "Human", "Raptor", "Small Bird", "Small Carnivore", "Ungulate", "Vehicle", "Wild Pig"]:
            sub_dirs.append(dirpath)
        
if sub_dirs == []:
    sub_dirs.append(dest_dir)

for i in sub_dirs:
    print(i)

if inp == "YES":
    ## LOAD MODELS
    print("Loading Models...")
    order_level_class_names = ["GIB", "Goat_Sheep", "Hare", "Human", "Raptor", "Small Bird", "Small Carnivore", "Ungulate", "Vehicle", "Wild Pig"]
    order_level_class_names.sort()
    ungulate_class_names = ["Camel", "Chinkara", "Nilgai", "Cattle"]
    ungulate_class_names.sort() 
    small_carnivores_class_names = ["Dog", "Desert Cat", "Fox"]
    small_carnivores_class_names.sort()
    
    model_load_start=time.time()
    if areModelsLoaded == False:
        order_level_model_path = os.path.join(os.getcwd(), r"Models\Refined_Hierarchical.ckpt")
        order_level_model = tf.keras.models.load_model(order_level_model_path)
        
        ungulate_model_path = os.path.join(os.getcwd(), r"Models\Efficient_Net_Ungulates_3.ckpt")
        ungulate_model = tf.keras.models.load_model(ungulate_model_path)   
        
        small_carnivore_model_path = os.path.join(os.getcwd(), r"Models\Efficient_Net_Small_Carnivores_1.ckpt")
        small_carnivore_model= tf.keras.models.load_model(small_carnivore_model_path)
        
        areModelsLoaded = True
        
    model_load_end = time.time()
    model_load_time = str(timedelta(seconds=round(model_load_end - model_load_start)))
    log.update({"Species Model Load Time" : model_load_time})
    print(model_load_time)
    
    for data_dir in sub_dirs:
        print()
        print(data_dir)
        ## CREATE LOGS
        
        log.update({"Run timestamp" : str(now)})
        # log.update({"GPU" : gpus})
        log.update({"GPU Available for Classification : " : gpu_name})
        # log.update({"CPU" : cpu})
        num_images = 0
        files = os.listdir(data_dir)
        for f in files:
            if f.lower().endswith(".jpg"):
                num_images += 1
        log.update({"Num images" : num_images})
        print(num_images)
        
        ## RUN MEGADETECTOR AND CREATE DETECTIONS.DF
        
        megadetector_start = time.time()
        json_dir, megadetector_log = megadetector(data_dir, num_images)
        if not megadetector_log == {}:
            log.update(megadetector_log)
        else:
            megadetector_end = time.time()
            megadetector_time = str(timedelta(seconds=round(megadetector_end - megadetector_start)))
            log.update({"Megadetector time" : megadetector_time})
            log.update({"Megadetector Filename" : os.path.basename(json_dir)})

        df_detections = get_detection_df(data_dir, json_dir)
        
        ## CROP IMAGES
        
        cropping_start = time.time() 
        cropped_images = os.path.join(data_dir,r"Cropped_images\*")
        cropped_dir = clean_path("\\".join(cropped_images.split("\\")[:-1]))
        if not os.path.exists(cropped_dir):
            print("Cropping Images")
            df_crop = df_detections.copy()
            df_crop["Cropped_image_directory"] = cropped_dir
            df_crop["Cropped_image_name"] = df_crop["Filename"] + "_" + df_crop["Detection_number"].astype(str) + ".jpg"
            df_crop["Cropped_image_path"] = (cropped_dir + "\\" + df_crop["Cropped_image_name"]).apply(clean_path)
            try:
                df_crop=crop_images_batch_gpu(df_crop,512)
            except:
                df_crop = crop_images_batch(df_crop,512)
                print(f"Cropping exception occured")
        else:
            df_crop = df_detections.copy()
            df_crop["Cropped_image_directory"] = cropped_dir
            df_crop["Cropped_image_name"] = df_crop["Filename"] + "_" + df_crop["Detection_number"].astype(str) + ".jpg"
            df_crop["Cropped_image_path"] = (cropped_dir + "\\" + df_crop["Cropped_image_name"]).apply(clean_path)
            print("Images already cropped...")
        cropping_end = time.time()
        cropping_time = str(timedelta(seconds=round(cropping_end - cropping_start)))
        log.update({"Cropping Time" : cropping_time})
        log.update({"Number of Detections" : len(df_detections)})
        
        ## ORDER LEVEL PREDICTIONS
        
        if os.path.exists(os.path.join(cropped_dir,r"Ungulate")) or os.path.exists(os.path.join(cropped_dir,r"Small Carnivore")) or os.path.exists(os.path.join(cropped_dir,r"Others")):
            df_order = df_crop.copy()
            df_order['Order_dir'] = np.nan
            df_order['Order_level_path'] = np.nan
            print("Order level classification already complete")
        else:
            order_level_start = time.time()
            print("Predicting Order Level Classes...")
            df_temp, num_cropped = predict_lower_level_species(data_dir, 
                                                               r"Cropped_images\*", 
                                                               order_level_class_names,
                                                               order_level_model,
                                                               level = "Order")
            
            order_level_end = time.time()
            df_order = pd.merge(df_crop, df_temp, on='Cropped_image_name', how='left')
            df_order["Order_pred"] = df_order["Order_pred"].fillna("Error")
            df_order["Order_dir"] = (cropped_dir + "\\" + df_order["Order_pred"]).apply(clean_path)
            df_order["Order_level_path"] = (df_order["Order_dir"] + "\\" + df_order["Cropped_image_name"]).apply(clean_path)
            
            unique_directories = set(df_order['Order_dir'])
            for directory in unique_directories:
                os.makedirs(directory, exist_ok=True)
            
            print("Moving Order Level Images...")
            move_images_batch(df_order["Cropped_image_path"], df_order["Order_level_path"])
            
            order_shift_end = time.time()
            order_pred_time = str(timedelta(seconds=round(order_level_end - order_level_start)))
            order_shift_time = str(timedelta(seconds=round(order_shift_end - order_level_end)))
            log.update({"Order Level Prediction Time" : order_shift_time})
            log.update({"Order Level Shifting Time" : order_shift_time})
        
        ## SMALL CARNIVORES PREDICT
        small_carnivores_start = time.time()
        if os.path.exists(os.path.join(cropped_dir,r"Small Carnivore")) and len(os.listdir(os.path.join(cropped_dir,r"Small Carnivore"))) > 0:
            print("Predicting Small Carnivores...")
            df_small_carnivore, num_small_carnivores = predict_lower_level_species(cropped_dir, 
                                                                                   r"Small Carnivore\*", 
                                                                                   small_carnivores_class_names,
                                                                                   small_carnivore_model,
                                                                                   level = "Species")
            df_small_carnivore["Order_pred"] = r"Small Carnivore"
            df_small_carnivore["Order_dir"] = (cropped_dir + "\\" + df_small_carnivore["Order_pred"]).apply(clean_path)
            df_small_carnivore["Order_level_path"] = (df_small_carnivore["Order_dir"] + "\\" + df_small_carnivore["Cropped_image_name"]).apply(clean_path)
            small_carnivores_end = time.time()
            small_carnivore_time = str(timedelta(seconds=round(small_carnivores_end - small_carnivores_start)))
            log.update({"Number of Small Carnivores Images" : num_small_carnivores})
            log.update({"Small Carnivore Model Pred Time" : small_carnivore_time})
        else:
            df_small_carnivore = pd.DataFrame(columns=['Cropped_image_name','Species_pred','Species_pred_prob'])
       
        ## UNGULATES PREDICT

        if os.path.exists(os.path.join(cropped_dir,r"Ungulate")) and len(os.listdir(os.path.join(cropped_dir,r"Ungulate"))) > 0 and not os.path.exists(os.path.join(cropped_dir,r"Cattle")):
            ungulate_start = time.time()
            print("Predicting Ungulates...")
            df_ungulate, num_ungulates = predict_lower_level_species(cropped_dir, 
                                                                     r"Ungulate\*", 
                                                                     ungulate_class_names,
                                                                     ungulate_model,
                                                                     level = "Species")
            df_ungulate["Order_pred"] = r"Ungulate"
            df_ungulate["Order_dir"] = (cropped_dir + "\\" + df_ungulate["Order_pred"]).apply(clean_path)
            df_ungulate["Order_level_path"] = (df_ungulate["Order_dir"] + "\\" + df_ungulate["Cropped_image_name"]).apply(clean_path)
            ungulate_end = time.time()
            ungulate_time = str(timedelta(seconds=round(ungulate_end - ungulate_start)))
            log.update({"Number of Ungulates Images" : num_ungulates})
            log.update({"Ungulate Model Pred Time" : ungulate_time})
        else:
            df_ungulate = pd.DataFrame(columns=['Cropped_image_name','Species_pred','Species_pred_prob'])
        
        species_shift_start = time.time()
        df_species = pd.concat([df_small_carnivore,df_ungulate])
        df_species["Species_dir"] = (cropped_dir + "\\" + df_species["Species_pred"]).apply(clean_path)
        df_species["Species_level_path"] = (df_species["Species_dir"] + "\\" + df_species["Cropped_image_name"]).apply(clean_path)
        
        #df_move = pd.merge(df_species, df_order, on='Cropped_image_name', how='left')
        try:
            df_move = df_species[df_species["Order_level_path"] != df_species["Species_level_path"]]
            unique_directories = set(df_move['Species_dir'])
            for directory in unique_directories:
                os.makedirs(directory, exist_ok=True)
            print("Moving Ungulates and Small Carnivores...")
            #copy_images_batch(df_move["Order_level_path"], df_move["Species_level_path"])
            #delete_images_batch(df_move["Order_level_path"])
            move_images_batch(df_move["Order_level_path"], df_move["Species_level_path"])
        except:
            print("No files to move")
            
        species_shift_end = time.time()
        species_shift_time = str(timedelta(seconds=round(species_shift_end - species_shift_start)))
        species_level_time = str(timedelta(seconds=round(species_shift_end - small_carnivores_start)))
        log.update({"Species Level Shift Imgs Time" : species_shift_time})
        log.update({"Species Level Predict and Shift" : species_level_time})
        
        ## SAVE FINAL PREDICTIONS.CSV
        if os.path.exists(os.path.join(data_dir,r"predictions.csv")):
            predictions = pd.read_csv(os.path.join(data_dir,r"predictions.csv"))
            preds = predictions.copy().dropna()
            preds_all = pd.concat([preds,df_species[~df_species["Cropped_image_name"].isin(preds["Cropped_image_name"].tolist())]])
            preds_all = preds_all[["Cropped_image_name","Species_pred","Species_pred_prob","Species_dir","Species_level_path"]]
            predictions.drop(columns = ["Species_pred","Species_pred_prob","Species_dir","Species_level_path"], inplace = True)
            df_final = pd.merge(predictions, preds_all, how='left', on = "Cropped_image_name")
        else:
            try:
                df_final = pd.merge(df_order, df_species.drop(columns=['Order_pred','Order_dir', 'Order_level_path']), on='Cropped_image_name', how='left')
            except:
                df_final = pd.merge(df_order, df_species, on='Cropped_image_name', how='left')
            df_final.drop(columns=['Order_dir', 'Order_level_path','Cropped_image_path'], inplace=True)
        df_final_path = os.path.join(data_dir, "predictions.csv")
        df_final.to_csv(df_final_path, index=False)
        
        ## SAVE LOG FILE
        print("Saving Logs")
        log_file_name = "_".join(data_dir.split("\\")[-3:])
        log_file_path = os.path.join(data_dir, f"{log_file_name}_log.json")
        with open(log_file_path, "w") as f:
            json.dump(log, f, indent=2)
        gc.collect()

Models loaded : True
GPU available
G:\Guzzler_data\Sorted_guzzler_data\2023\2023_1\CameraTrapPhotos_2023_1\RKVY\GIB\20230129_113814__to__20230201_090348
G:\Guzzler_data\Sorted_guzzler_data\2023\2023_1\CameraTrapPhotos_2023_1\RKVY\GIB\20230201_090351__to__20230201_152711
G:\Guzzler_data\Sorted_guzzler_data\2023\2023_1\CameraTrapPhotos_2023_1\RKVY\GIB\20230201_152713__to__20230203_130946
G:\Guzzler_data\Sorted_guzzler_data\2023\2023_1\CameraTrapPhotos_2023_1\RKVY\GIB\20230203_130947__to__20230203_232908
G:\Guzzler_data\Sorted_guzzler_data\2023\2023_1\CameraTrapPhotos_2023_1\RKVY\GIB\20230203_232909__to__20230219_122053
G:\Guzzler_data\Sorted_guzzler_data\2023\2023_1\CameraTrapPhotos_2023_1\RKVY\GIB\20230219_122053__to__20230219_214647
G:\Guzzler_data\Sorted_guzzler_data\2023\2023_1\CameraTrapPhotos_2023_1\RKVY\GIB\20230219_214648__to__20230222_002453
G:\Guzzler_data\Sorted_guzzler_data\2023\2023_1\CameraTrapPhotos_2023_1\RKVY\GIB\20230222_002454__to__20230222_125227
G:\Guzzler_data\Sorte

Copying images: 0it [00:04, ?it/s]
Removing source images: 0it [00:01, ?it/s]

All 0 images copied and removed at 2024-03-07 12:49:19.991630


Saving Logs

G:\Guzzler_data\Sorted_guzzler_data\2023\2023_1\CameraTrapPhotos_2023_1\RKVY\GIB\20230201_090351__to__20230201_152711
10000
Megadetector model
Megadetector output file already exists.. Going for species classification
Saving detections at G:\Guzzler_data\Sorted_guzzler_data\2023\2023_1\CameraTrapPhotos_2023_1\RKVY\GIB\20230201_090351__to__20230201_152711\RKVY_GIB_20230201_090351__to__20230201_152711_megadetector.json...
Generating detections.csv...
Images already cropped...
Order level classification already complete
Predicting Small Carnivores...
5/5 [==============================] - 3s 771ms/step
Moving Ungulates and Small Carnivores...


Copying images: 0it [00:01, ?it/s]
Removing source images: 0it [00:01, ?it/s]

...

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [35:17<00:00, 21.17s/it]


Bounding Boxes Created
Generating detections.csv...
Images already cropped...
Order level classification already complete
Predicting Small Carnivores...
2/2 [==============================] - 2s 2s/step
Moving Ungulates and Small Carnivores...


Copying images: 0it [00:01, ?it/s]
Removing source images: 0it [00:01, ?it/s]

All 0 images copied and removed at 2024-03-07 22:38:16.859448


Saving Logs
